# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 13 2022 12:41PM,246310,10,8521202,Beach Products Inc,Released
1,Sep 13 2022 12:41PM,246310,10,8521203,Beach Products Inc,Released
2,Sep 13 2022 12:41PM,246310,10,8521204,Beach Products Inc,Released
3,Sep 13 2022 12:41PM,246310,10,8521205,Beach Products Inc,Released
4,Sep 13 2022 12:41PM,246308,12,HH-36895.1,Hush Hush,Released
5,Sep 13 2022 12:41PM,246308,12,HH-36897.1,Hush Hush,Released
6,Sep 13 2022 12:41PM,246308,12,HH-36898.1,Hush Hush,Released
7,Sep 13 2022 12:41PM,246308,12,HH-36899.1,Hush Hush,Released
8,Sep 13 2022 12:41PM,246309,10,HH-36896.2,Hush Hush,Released
9,Sep 13 2022 12:39PM,246307,10,0085977764,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,246306,Released,5
27,246307,Released,8
28,246308,Released,4
29,246309,Released,1
30,246310,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246306,NaN,NaN,5.0
246307,NaN,NaN,8.0
246308,NaN,NaN,4.0
246309,NaN,NaN,1.0
246310,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246064,0.0,0.0,1.0
246190,1.0,0.0,1.0
246224,0.0,0.0,1.0
246226,0.0,1.0,0.0
246227,22.0,38.0,59.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246064,0,0,1
246190,1,0,1
246224,0,0,1
246226,0,1,0
246227,22,38,59


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246064,0,0,1
1,246190,1,0,1
2,246224,0,0,1
3,246226,0,1,0
4,246227,22,38,59


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246064,,,1
1,246190,1,,1
2,246224,,,1
3,246226,,1,
4,246227,22,38,59


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 13 2022 12:41PM,246310,10,Beach Products Inc
4,Sep 13 2022 12:41PM,246308,12,Hush Hush
8,Sep 13 2022 12:41PM,246309,10,Hush Hush
9,Sep 13 2022 12:39PM,246307,10,ISDIN Corporation
17,Sep 13 2022 12:39PM,246306,10,Emerginnova
22,Sep 13 2022 12:29PM,246300,19,"AdvaGen Pharma, Ltd"
58,Sep 13 2022 12:16PM,246297,10,Eywa Pharma Inc.
59,Sep 13 2022 12:15PM,246295,19,ACG North America LLC
61,Sep 13 2022 12:13PM,246293,10,Ivaoes Animal Health
62,Sep 13 2022 12:05PM,246292,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 13 2022 12:41PM,246310,10,Beach Products Inc,,,4
1,Sep 13 2022 12:41PM,246308,12,Hush Hush,,,4
2,Sep 13 2022 12:41PM,246309,10,Hush Hush,,,1
3,Sep 13 2022 12:39PM,246307,10,ISDIN Corporation,,,8
4,Sep 13 2022 12:39PM,246306,10,Emerginnova,,,5
5,Sep 13 2022 12:29PM,246300,19,"AdvaGen Pharma, Ltd",,,36
6,Sep 13 2022 12:16PM,246297,10,Eywa Pharma Inc.,,,1
7,Sep 13 2022 12:15PM,246295,19,ACG North America LLC,,,2
8,Sep 13 2022 12:13PM,246293,10,Ivaoes Animal Health,,,1
9,Sep 13 2022 12:05PM,246292,10,ISDIN Corporation,,7,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 12:41PM,246310,10,Beach Products Inc,4,,
1,Sep 13 2022 12:41PM,246308,12,Hush Hush,4,,
2,Sep 13 2022 12:41PM,246309,10,Hush Hush,1,,
3,Sep 13 2022 12:39PM,246307,10,ISDIN Corporation,8,,
4,Sep 13 2022 12:39PM,246306,10,Emerginnova,5,,
5,Sep 13 2022 12:29PM,246300,19,"AdvaGen Pharma, Ltd",36,,
6,Sep 13 2022 12:16PM,246297,10,Eywa Pharma Inc.,1,,
7,Sep 13 2022 12:15PM,246295,19,ACG North America LLC,2,,
8,Sep 13 2022 12:13PM,246293,10,Ivaoes Animal Health,1,,
9,Sep 13 2022 12:05PM,246292,10,ISDIN Corporation,5,7,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 12:41PM,246310,10,Beach Products Inc,4,,
1,Sep 13 2022 12:41PM,246308,12,Hush Hush,4,,
2,Sep 13 2022 12:41PM,246309,10,Hush Hush,1,,
3,Sep 13 2022 12:39PM,246307,10,ISDIN Corporation,8,,
4,Sep 13 2022 12:39PM,246306,10,Emerginnova,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 12:41PM,246310,10,Beach Products Inc,4.0,NaN,NaN
1,Sep 13 2022 12:41PM,246308,12,Hush Hush,4.0,NaN,NaN
2,Sep 13 2022 12:41PM,246309,10,Hush Hush,1.0,NaN,NaN
3,Sep 13 2022 12:39PM,246307,10,ISDIN Corporation,8.0,NaN,NaN
4,Sep 13 2022 12:39PM,246306,10,Emerginnova,5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 12:41PM,246310,10,Beach Products Inc,4.0,0.0,0.0
1,Sep 13 2022 12:41PM,246308,12,Hush Hush,4.0,0.0,0.0
2,Sep 13 2022 12:41PM,246309,10,Hush Hush,1.0,0.0,0.0
3,Sep 13 2022 12:39PM,246307,10,ISDIN Corporation,8.0,0.0,0.0
4,Sep 13 2022 12:39PM,246306,10,Emerginnova,5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3201682,58.0,16.0,0.0
12,246308,4.0,0.0,0.0
15,246227,59.0,38.0,22.0
19,1231305,41.0,1.0,1.0
20,246267,19.0,11.0,0.0
21,492290,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3201682,58.0,16.0,0.0
1,12,246308,4.0,0.0,0.0
2,15,246227,59.0,38.0,22.0
3,19,1231305,41.0,1.0,1.0
4,20,246267,19.0,11.0,0.0
5,21,492290,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,58.0,16.0,0.0
1,12,4.0,0.0,0.0
2,15,59.0,38.0,22.0
3,19,41.0,1.0,1.0
4,20,19.0,11.0,0.0
5,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,58.0
1,12,Released,4.0
2,15,Released,59.0
3,19,Released,41.0
4,20,Released,19.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,0.0,0.0,22.0,1.0,0.0,0.0
Executing,16.0,0.0,38.0,1.0,11.0,1.0
Released,58.0,4.0,59.0,41.0,19.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,22.0,1.0,0.0,0.0
1,Executing,16.0,0.0,38.0,1.0,11.0,1.0
2,Released,58.0,4.0,59.0,41.0,19.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,22.0,1.0,0.0,0.0
1,Executing,16.0,0.0,38.0,1.0,11.0,1.0
2,Released,58.0,4.0,59.0,41.0,19.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()